In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [0]:
def area(box):
    return (box[2] - box[0]) * (box[3] - box[1])


def intersection_over_union(boxes):
    assert(len(boxes) == 8)
    boxA = boxes[:4].values
    boxB = boxes[4:].values
    
    boxAArea = area(boxA)
    boxBArea = area(boxB)
    
    if (boxAArea == 0 or boxBArea == 0):
        return 0
        
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)

    
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

In [3]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [4]:
votes = pd.read_csv("drive/My Drive/sna9/train3_data.csv", index_col=0)
votes.head()

,itemId,Xmin,Ymin,Xmax,Ymax,Xmin_true,Ymin_true,Xmax_true,Ymax_true,acc,se_Xmin,se_Ymin,se_Xmax,se_Ymax,Y_xmin,Y_ymin,Y_xmax,Y_ymax,L,H,P,S,L/H,D,iou,best
userId,,,,,,,,,,,,,,,,,,,,,,,,,,
1164,6132,62,656,276,803,47,650,213,812,0.988892,225,36,3969,81,1,1,0,1,214.0,147.0,722.0,31458.0,1.455782,259.624729,0.613974,1
1399,6132,184,662,235,740,47,650,213,812,0.936663,18769,144,484,5184,0,0,1,0,51.0,78.0,258.0,3978.0,0.653846,93.193347,0.079069,0
1694,6132,63,648,272,830,47,650,213,812,0.989526,256,4,3481,324,1,1,0,1,209.0,182.0,782.0,38038.0,1.148352,277.137150,0.598080,1
916,6132,111,637,253,831,47,650,213,812,0.983958,4096,169,1600,361,0,0,1,1,142.0,194.0,672.0,27548.0,0.731959,240.416306,0.435805,0
1621,6132,60,651,242,819,47,650,213,812,0.997269,169,1,841,49,1,1,1,1,182.0,168.0,700.0,30576.0,1.083333,247.685284,0.750206,1


### Вначале выберу лучших по iou

In [0]:
#votes["iou"] = votes[['Xmin','Ymin','Xmax','Ymax','Xmin_true','Ymin_true','Xmax_true','Ymax_true']].apply(intersection_over_union, axis=1)
#votes.to_csv("drive/My Drive/sna9/train3_data.csv")

In [6]:
votes["iou"].describe()

count    5320.000000
mean        0.454321
std         0.193210
min         0.000000
25%         0.339031
50%         0.475980
75%         0.582605
max         1.000000
Name: iou, dtype: float64

In [0]:
for i in range(40, 90, 1):
    votes = votes[votes.iou > i/100] # ну, от балды просто
    print('i = ',i/100, ' shape - ', votes.shape, ' items - ', len(votes.itemId.unique()))
    
    Xminmin = votes.groupby("itemId")['Xmin'].min().reset_index().set_index('itemId')
    Yminmin = votes.groupby("itemId")['Ymin'].min().reset_index().set_index('itemId')
    Xmaxmax = votes.groupby("itemId")['Xmax'].max().reset_index().set_index('itemId')
    Ymaxmax = votes.groupby("itemId")['Ymax'].max().reset_index().set_index('itemId')
    
    maxbox = pd.concat([Xminmin,Yminmin,Xmaxmax,Ymaxmax], axis=1)
    trubox = votes.groupby("itemId")['Xmin_true','Ymin_true','Xmax_true','Ymax_true'].min().reset_index().set_index('itemId')
    data = pd.concat([maxbox, trubox], axis=1)
    
    data["iou"] = data.apply(intersection_over_union, axis=1)
    print(data["iou"].mean())

Выберу как лучший вариант i =  0.47  shape -  (2751, 26)  items -  906
0.6126846331657105  потомушта примерно пол-набора и объекты почти все

In [50]:
#votes['best'] = 0
#votes['best'][votes.iou > 0.47] = 1
votes.best.sum()

2751

In [48]:
votes.shape

(5320, 25)

In [11]:
len(votes.itemId.unique()) # 941 - all

941

## Select extrim coords

In [0]:
Xminmin = votes.groupby("itemId")['Xmin'].min().reset_index().set_index('itemId')
Yminmin = votes.groupby("itemId")['Ymin'].min().reset_index().set_index('itemId')
Xmaxmax = votes.groupby("itemId")['Xmax'].max().reset_index().set_index('itemId')
Ymaxmax = votes.groupby("itemId")['Ymax'].max().reset_index().set_index('itemId')

In [6]:
maxbox = pd.concat([Xminmin,Yminmin,Xmaxmax,Ymaxmax], axis=1)
maxbox.head()

,Xmin,Ymin,Xmax,Ymax
itemId,,,,
5,151,914,511,1331
10,72,559,427,858
13,46,600,621,981
30,65,353,324,491
32,9,279,388,884


In [0]:
trubox = votes.groupby("itemId")['Xmin_true','Ymin_true','Xmax_true','Ymax_true'].min().reset_index().set_index('itemId')
trubox.head()

In [0]:
data = pd.concat([maxbox, trubox], axis=1)
data.head()

In [0]:
data["iou"] = data.apply(intersection_over_union, axis=1)

In [10]:
data["iou"].mean()

0.5822325332074827

In [0]:
data["iou"].describe()

count    941.000000
mean       0.582233
std        0.147541
min        0.079091
25%        0.489063
50%        0.571360
75%        0.673746
max        1.000000
Name: iou, dtype: float64

In [0]:
data.sort_index().to_csv("drive/My Drive/sna9/dataiou.csv")

# Expanse borders

1.   Уменьшение box на 1% - 0.5519905825727914
2.  Увеличение всех на 1% - 0.562786686942409
3. Уменьшить правый нижний угол 0.5631181862644539
4. Увеличить -//-0.5691904647663387
5. Уменьшить правый верхний угол 0.5647795294467304
6. Увеличить -//- 0.5700939982942403
7. Уменьшить левый нижний угол 0.5632979914923907
8. Увеличить -//- 0.5697250207874525
9. Уменьшить левый верхний угол 0.5650297425732911
10. Увеличить -//- 0.5706679363722884



# СДВИГИ


1.   Расширить правую границу 0.5816498787222311
2.   Сузить правую границу 0.5817036727054256
3.   Расширить левую границу 0.5822091124929779
4.   Сузить левую границу  0.5816593765765847
5.   Расширить обе границы 0.5816806121692282
6.   Сузить обе границы 0.5811535433529459

1.   Расширить верхнюю границу 0.570543789440679
2.   Сузить верхнюю границу 0.5654474285794497
3.   Расширить нижнюю границу 0.569603477531477
4.   Сузить нижнюю границу  0.5636927549358063
5.   Расширить обе границы 0.5630784627480657
6.   Сузить обе границы 0.5529732894591048


*  Сдвинуть влево-вверх 0.5572311578440731
* Сдвинуть влево-вниз 0.5574411608886817
* Сдвинуть вправо-вверх 0.5567795927088312
* Сдвинуть вправо-вниз 0.5572509602623342





In [0]:
d1 = data['Xmin'].apply(lambda z: z*1.01)
d2 = data['Ymin'].apply(lambda z: z*0.99)
d3 = data['Xmax'].apply(lambda z: z*1.01)
D = data['Ymax'].apply(lambda z: z*.99)
d4 = data[[ 'Xmin_true','Ymin_true','Xmax_true','Ymax_true']]
pd.concat([d1, d2, d3, D, d4], axis=1).head()

,Xmin,Ymin,Xmax,Ymax,Xmin_true,Ymin_true,Xmax_true,Ymax_true
itemId,,,,,,,,
5,152.51,904.86,516.11,1317.69,198,827,649,1340
10,72.72,553.41,431.27,849.42,26,605,433,847
13,46.46,594.00,627.21,971.19,233,698,622,920
30,65.65,349.47,327.24,486.09,61,368,329,530
32,9.09,276.21,391.88,875.16,30,536,328,813


In [0]:
data["iou"] = pd.concat([d1, d2, d3, D, d4], axis=1).apply(intersection_over_union, axis=1)
data["iou"].mean()

0.5572509602623342

In [0]:
data.iou.describe()

count    941.000000
mean       0.569190
std        0.153183
min        0.070829
25%        0.475512
50%        0.564101
75%        0.663517
max        0.975968
Name: iou, dtype: float64

# TRY IT...

In [0]:
votest = pd.read_csv("drive/My Drive/sna9/test1_data.csv", index_col=0)

In [0]:
Xminmin = votest.groupby("itemId")['Xmin'].min().reset_index().set_index('itemId')
Yminmin = votest.groupby("itemId")['Ymin'].min().reset_index().set_index('itemId')
Xmaxmax = votest.groupby("itemId")['Xmax'].max().reset_index().set_index('itemId')
Ymaxmax = votest.groupby("itemId")['Ymax'].max().reset_index().set_index('itemId')

In [0]:
maxbox = pd.concat([Xminmin,Yminmin,Xmaxmax,Ymaxmax], axis=1)
maxbox.head()

,Xmin,Ymin,Xmax,Ymax
itemId,,,,
18,126,634,764,1132
19,27,559,119,729
33,21,380,414,652
62,35,824,639,1262
114,41,570,216,780


In [0]:
m1 = maxbox['Xmin'].apply(lambda z: z*.99)
m2 = maxbox[['Ymin','Xmax']]#.apply(lambda z: z*.99)
m3 = maxbox['Ymax']#.apply(lambda z: z*1.01)
pd.concat([m1, m2, m3], axis=1).head()

,Xmin,Ymin,Xmax,Ymax
itemId,,,,
18,124.74,634,764,1132
19,26.73,559,119,729
33,20.79,380,414,652
62,34.65,824,639,1262
114,40.59,570,216,780


In [0]:
pd.concat([m1, m2, m3], axis=1).sort_index().to_csv("drive/My Drive/sna9/mbLBplus.csv", header=None)

## ДОБАВЛЮ ДАННЫЕ О МЕСТЕ ОБЪЕКТОВ ВНУТРИ ОБЩЕЙ КАРТИНКИ

In [0]:
data.head()


,Xmin,Ymin,Xmax,Ymax,Xmin_true,Ymin_true,Xmax_true,Ymax_true,iou
itemId,,,,,,,,,
5,151,914,511,1331,198,827,649,1340,0.525002
10,72,559,427,858,26,605,433,847,0.736209
13,46,600,621,981,233,698,622,920,0.394233
30,65,353,324,491,61,368,329,530,0.640055
32,9,279,388,884,30,536,328,813,0.360035


In [0]:
# Самая нижняя точка
data.Ymin.min()  # 40
data.Ymin_true.min() #21  !!!

21

Тьфу!! нижняя по орпеделению 0-0... т.е. Xmin=Ymin=0

In [0]:
# Самая верхняя точка
data.Ymax.max()  # 1903
data.Ymax_true.max() #1949  !!!

""" Ок. Ymax пусть будет 2000 """

1949

In [0]:
# Самая правая точка
data.Xmax.max()  # 1051
#data.Xmax_true.max() #1061  !!!

""" Ок. Xmax пусть будет 1100 
КАК ИНТЕРЕСНО! Картинка вытянута вверх """

1051